In [2]:
import json
from opensearchpy import OpenSearch
from pprint import pprint

host = 'localhost'
port = 9200
base_url = "https://{}:{}/".format(host, port)
auth = ('admin', 'admin')  # For testing only. Don't store credentials in code.
index_name = "bbuy_products"
client = OpenSearch(
    hosts=[{'host': host, 'port': port}],
    http_compress=True,  # enables gzip compression for request bodies
    http_auth=auth,
    # client_cert = client_cert_path,
    # client_key = client_key_path,
    use_ssl=True,
    verify_certs=False,
    ssl_assert_hostname=False,
    ssl_show_warn=False,
)

In [3]:
index_name = 'bbuy_queries'

In [8]:

query = {
    "size": 0,
    "aggs": {
        "Query": {
            "terms": {
                "size": 10,
                "field": "query.keyword"
            },
            "aggs": {
                "Docs": {
                    "terms": {
                        "size": 5,
                        "field": "sku.keyword"
                    }
                }
            }
        }
    }
}

client.search(query, index_name)


{'took': 1235,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': None,
  'hits': []},
 'aggregations': {'Query': {'doc_count_error_upper_bound': 0,
   'sum_other_doc_count': 1797068,
   'buckets': [{'key': 'lcd tv',
     'doc_count': 16522,
     'Docs': {'doc_count_error_upper_bound': 0,
      'sum_other_doc_count': 1321,
      'buckets': [{'key': '2620821', 'doc_count': 7169},
       {'key': '2138389', 'doc_count': 3575},
       {'key': '2893174', 'doc_count': 2610},
       {'key': '1831054', 'doc_count': 1260},
       {'key': '2047641', 'doc_count': 587}]}},
    {'key': '2622037 2127204 2127213 2121716 2138291',
     'doc_count': 8531,
     'Docs': {'doc_count_error_upper_bound': 0,
      'sum_other_doc_count': 6365,
      'buckets': [{'key': '2138291', 'doc_count': 654},
       {'key': '2622037', 'doc_count': 475},
       {'key': '2127213', 'doc_count': 432},
       {'key'

We have not performed any normalization on the queries: “Hp touchpad” and “hp touchpad” are distinct queries, as are “iPad” and “ipad”. In general, we want to use an analyzer to normalize queries, but sometimes it’s useful to see the raw queries. For example, we can explore whether users are typing their search queries (probably in lowercase without punctuation) or copying and pasting them from product titles on other sites (which probably show up in title case and often include punctuation).

There are some strange-looking queries! It’s unlikely that thousands of users typed “LaborDay_Computers_20110902” into the search box. The more likely explanation is that the search application generated this query automatically – Perhaps the users clicked on a promotional banner.

We’re performing a nested aggregation, something we didn’t cover last week. The outer aggregation is on the query field, grouping searches by query, while the inner aggregation groups each of these buckets by SKU. As a result, we see the top-clicked SKUs for each query. This is a useful approach for getting a high-level view of searcher behavior.

Perhaps the strangest query in the list is “2622037 2127204 2127213 2121716 2138291.”  Is it spam? Let’s investigate by searching the queries for this text:

In [13]:
query = {"query": {"match_all": {}}, "rescore": [{"query": {"rescore_query": {"match_phrase": {"body": {"query": "Fox jumped"}}}, "query_weight": 1.0, "rescore_query_weight": 2.0}, "window_size": 2}, 
{    
    "query": {"rescore_query": {"function_score": {"field_value_factor": {"field": "price", "missing": 1}}}, "query_weight": 1.0, "rescore_query_weight": 4.0}, "window_size": 1}]}

client.search(query, index_name)


{'took': 28,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': 5.0,
  'hits': [{'_index': 'bbuy_queries',
    '_type': '_doc',
    '_id': 'hyXci4EBzRqzEMNiLRB9',
    '_score': 5.0,
    '_source': {'user': '2629a01969f5da7d183c377dd962b3f324d4f351',
     'sku': 2629464,
     'category': 'pcmcat143700050032',
     'query': 'iPhone Av cable',
     'click_time': '2011-09-07T12:16:04.446000',
     'query_time': '2011-09-07T12:15:39.458000'}},
   {'_index': 'bbuy_queries',
    '_type': '_doc',
    '_id': 'iCXci4EBzRqzEMNiLRB9',
    '_score': 1.0,
    '_source': {'user': '2629a01969f5da7d183c377dd962b3f324d4f351',
     'sku': 2614056,
     'category': 'pcmcat214700050000',
     'query': 'iPhone 4 case',
     'click_time': '2011-09-08T19:14:08.767000',
     'query_time': '2011-09-08T19:10:50.207000'}},
   {'_index': 'bbuy_queries',
    '_type': '_doc',
    '_id': 'iSXci4EBzRqzEMNiLRB

In [9]:

query =  {
                "query": {
                    "match_all": {}
                },
                "rescore": [
                    {
                        "query": {
                            "rescore_query": {
                                "function_score": {

                                    "script_score": {
                                        "script": {
                                            "source": """
                                                        if (doc['id.keyword'].value == "doc_a"){
                                                        return doc.price.value;
                                                        }
                                                        return 1;
                                                        """
                                        }
                                    }
                                }
                            },
                            "query_weight": 1.0,
                            "rescore_query_weight": 1.0
                        },
                        "window_size": 10
                    }
                ]
            }


In [8]:
client.search(query, index_name)

RequestError: RequestError(400, 'search_phase_execution_exception', 'runtime error')